# Função para predição do valor de fechamento 

In [4]:
"""
Exemplo de dados_entrada:
dados_entrada = {
    "Date": "2024-11-27",
    "Open": 190.75,
    "High": 195.10,
    "Low": 189.50,
    "Volume": 72000000
}
"""

import torch
import numpy as np
import pandas as pd
import torch.nn as nn


hidden_size = 128
num_layers = 2
learning_rate = 0.005130547232547332,
batch_size = 32
num_epochs = 43


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm1(x, (h0, c0))
        out = self.fc1(out[:, -1, :])
        out, _ = self.lstm2(x, (h0, c0))
        out = self.fc2(out[:, -1, :])
        return out
    
    
def valor_fechamento(dados_entrada):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTM(11, hidden_size, num_layers, 1).to(device)
    model.load_state_dict(torch.load('modelo_aapl_lstm.pth'))
    model.eval()
    scaler = torch.load('scaler.pkl')
    
    # estabelecimento de DF com os dados de entrada, normalização e criação das features extras
    df = pd.DataFrame([dados_entrada])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Weekday'] = df['Date'].dt.weekday
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['day_sin'] = np.sin(2 * np.pi * df['Date'].dt.dayofyear / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['Date'].dt.dayofyear / 365)
    cols_norm = ['Open', 'High', 'Low', 'Volume', 'Weekday', 'Month', 'Year']
    dados_normalizados = df[cols_norm + ['day_sin', 'day_cos']].copy()
    dados_normalizados[cols_norm] = scaler.transform(dados_normalizados[cols_norm])
    
    # predição de Close
    X = torch.tensor(dados_normalizados.values, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        predicao_normalizada = model(X).item()
    
    # desnormalizar o valor de Close
    min_close, max_close = scaler.data_min_[3], scaler.data_max_[3]  # 'Closer' é o índice 3
    predicao_real = predicao_normalizada * (max_close - min_close) + min_close
    
    print(f"Valor previsto de 'Close': {predicao_real:.2f}")
    return predicao_real


### Teste

In [5]:
dados_entrada = {
    "Date": "2024-11-27",
    "Open": 190.75,
    "High": 195.10,
    "Low": 189.50,
    "Volume": 72000000
}
valor_fechamento(dados_entrada)


C:\Users\vitor\AppData\Local\Temp\ipykernel_11212\2878759728.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modelo_aapl_lstm.pth'))


RuntimeError: Error(s) in loading state_dict for LSTM:
	size mismatch for lstm1.weight_ih_l0: copying a param with shape torch.Size([512, 9]) from checkpoint, the shape in current model is torch.Size([512, 11]).
	size mismatch for lstm2.weight_ih_l0: copying a param with shape torch.Size([512, 9]) from checkpoint, the shape in current model is torch.Size([512, 11]).